# Joint Bert Model for slot and intent classification

### Imports

In [2]:
import json
import os
import re
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import AutoTokenizer
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

c:\Users\falkt\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [4]:
class RawData(object):
    def __init__(self, id, intent, text, slots, memory):
        self.id = id
        self.intent = intent
        self.text = text
        self.slots = slots
        self.memory = memory

    def __repr__(self):
        return str(json.dumps(self.__dict__, indent=2))


"""
reads json from data file
returns a list containing DataInstance objects
"""


def read_train_json_file(filename):
    if os.path.exists(filename):
        intents = []

        with open(filename, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)

            for k in data.keys():
                intent = data[k]["intent"]
                text = data[k]["text"]
                slots = data[k]["slots"]
                memory = data[k]["memory"]

                temp = RawData(k, intent, text, slots, memory)
                intents.append(temp)

        return intents
    else:
        raise FileNotFoundError("No file found with that path!")

# read from json file
data = read_train_json_file("JERTmate_data.json")

KeyError: 'remove'

### Encode Intents

In [9]:
encoded_intents = [d.intent for d in data]

### Encode Slots

In [10]:
def encode_slots(data):
    slots = []
    phantom_slot = None
    for slot in data:
        if len(slot) > 3:
            phantom_slot = slot[0:3]
        else:
            slots.extend(slot)

    # Add padding
    slots.extend(np.zeros(60 - len(slots)))

    if phantom_slot is not None:
        slots.extend(phantom_slot)
    else:
        slots.extend(np.zeros(3))
        
    return slots

encoded_slots = []
for data_point in data:
    encoded_slots.append(encode_slots(data_point.slots))

### Load Tokenizer and Encode Inputs

In [3]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#encode and pad texts
def encode_texts(tokenizer, data):
    texts = []
    for data_point in data:
        text = ''
        for previous_sentence in data_point.memory:
            text += previous_sentence.text + " [SEP] "

        text += data_point.text
        texts.append(text)
        
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
encoded_texts = encode_texts(tokenizer, data)

#parse texts and memory into input data
encoded_inputs = []
for i in range(len(data)):
    intent_memory = []
    slot_memory = []

    for sentence in data[i].memory:
        intent_memory.append(sentence.intent)
        slot_memory.extend(encode_slots(sentence.slots))

    encoded_inputs.append([encoded_texts[i], intent_memory, slot_memory])

NameError: name 'data' is not defined

### Split Data - Train 80% | Validation 10% | Test 10%

In [11]:
def split_data_by_intent(encoded_intents, train_ratio, val_ratio, test_ratio, seed=42):
    np.random.seed(seed)
    intent_indices = defaultdict(list)

    # Group indices by intent
    for idx, intent in enumerate(encoded_intents):
        intent_indices[intent.numpy()].append(idx)

    train_indices = []
    val_indices = []
    test_indices = []

    # Split indices for each intent
    for indices in intent_indices.values():
        np.random.shuffle(indices)
        n_total = len(indices)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        
        train_indices.extend(indices[:n_train])
        val_indices.extend(indices[n_train:n_train + n_val])
        test_indices.extend(indices[n_train + n_val:])

    return train_indices, val_indices, test_indices

def split_and_shuffle(encoded_texts, encoded_slots, encoded_intents, train_ratio, val_ratio, test_ratio):
    train_indices, val_indices, test_indices = split_data_by_intent(encoded_intents, train_ratio, val_ratio, test_ratio)

    def extract_data(indices):
        return {key: tf.gather(value, indices) for key, value in encoded_texts.items()}, tf.gather(encoded_slots, indices), tf.gather(encoded_intents, indices)

    train_encoded_texts, train_encoded_slots, train_encoded_intents = extract_data(train_indices)
    val_encoded_texts, val_encoded_slots, val_encoded_intents = extract_data(val_indices)
    test_encoded_texts, test_encoded_slots, test_encoded_intents = extract_data(test_indices)

    return (train_encoded_texts, train_encoded_slots, train_encoded_intents), (val_encoded_texts, val_encoded_slots, val_encoded_intents), (test_encoded_texts, test_encoded_slots, test_encoded_intents)

(train_encoded_texts, train_encoded_slots, train_encoded_intents), (val_encoded_texts, val_encoded_slots, val_encoded_intents), (test_encoded_texts, test_encoded_slots, test_encoded_intents) = split_and_shuffle(encoded_inputs, encoded_slots, encoded_intents, 0.8, 0.1, 0.1)

### Define Model

In [26]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None, model_name=model_name, dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels, name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels, name="slot_classifier")

    def __call__(self, inputs, **kwargs):
        # two outputs from BERT
        trained_bert = self.bert(inputs[0], **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state

        # slot_filling / classification
        slot_input = tf.concat([sequence_output, tf.reshape(inputs[1:], [-1])], axis=-1) # Combine sequence output and all memory data

        slot_input = self.dropout(slot_input, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(slot_input)

        # intent classification
        intent_input = tf.concat([pooled_output, inputs[1]], axis=-1) # Combine pooled output and memory intent data

        intent_input = self.dropout(intent_input, training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(intent_input)

        return slot_logits, intent_logits, sequence_output

    def get_config(self):
        config = super(JointIntentAndSlotFillingModel, self).get_config()
        config.update({
            "dropout": self.dropout_prob,
            "intent_num_labels": self.intent_num_labels,
            "slot_num_labels": self.slot_num_labels,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
joint_model = JointIntentAndSlotFillingModel(intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Train and Compile

In [13]:
# optimizer
opt = Adam(learning_rate=5e-5, epsilon=1e-08)

# learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 13:
        return lr
    else:
        return lr * 0.97
learning_rate_callback = LearningRateScheduler(scheduler)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",  # Save the model with the epoch number in the filename
    save_freq='epoch',
    verbose=2 # Save the model every 5 epochs
)

# two losses, one for slots, another for intents
losses = [SparseCategoricalCrossentropy(from_logits=True, name="slot_loss"), SparseCategoricalCrossentropy(from_logits=True, name="intent_loss")]
metrics = [SparseCategoricalAccuracy(name="slot_accuracy"), SparseCategoricalAccuracy(name="intent_accuracy")]

# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

history = joint_model.fit(
    x=train_encoded_texts, 
    y=(train_encoded_slots, train_encoded_intents), 
    validation_data=(val_encoded_texts, (val_encoded_slots, val_encoded_intents)), 
    epochs=20, 
    batch_size=32,
    shuffle=True, 
    callbacks=[learning_rate_callback, checkpoint_callback],
)

Epoch 1/20
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.3142 - loss: 2.5271 - slot_accuracy: 0.9288
Epoch 1: saving model to model_epoch_01.keras
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 7231s 3s/step - intent_accuracy: 0.3142 - loss: 2.5270 - slot_accuracy: 0.9288 - val_intent_accuracy: 0.4787 - val_loss: 1.7735 - val_slot_accuracy: 0.9645 - learning_rate: 5.0000e-05
Epoch 2/20


c:\Users\falkt\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_api.py:102: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


2776/2776 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.5354 - loss: 1.6793 - slot_accuracy: 0.9663
Epoch 2: saving model to model_epoch_02.keras
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 7147s 3s/step - intent_accuracy: 0.5354 - loss: 1.6793 - slot_accuracy: 0.9663 - val_intent_accuracy: 0.6088 - val_loss: 1.4713 - val_slot_accuracy: 0.9712 - learning_rate: 5.0000e-05
Epoch 3/20
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.6301 - loss: 1.4190 - slot_accuracy: 0.9724
Epoch 3: saving model to model_epoch_03.keras
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 7028s 3s/step - intent_accuracy: 0.6301 - loss: 1.4190 - slot_accuracy: 0.9724 - val_intent_accuracy: 0.6665 - val_loss: 1.2910 - val_slot_accuracy: 0.9754 - learning_rate: 5.0000e-05
Epoch 4/20
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.6813 - loss: 1.2516 - slot_accuracy: 0.9760
Epoch 4: saving model to model_epoch_04.keras
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 7001s 3s/step - intent_accuracy: 0.6813 - loss: 1.2516 - s

### Evaluate Model

keep in mind -> test slot accuracy will be higher than reality.
        Because 90% of the data points are 0, it can just guess 0 and be right 85% of the time

In [30]:
test_loss, test_slot_acc, test_intent_acc = joint_model.evaluate(x=test_encoded_texts, y=(test_encoded_slots, test_encoded_intents), batch_size=32)

print(f"Test Slot Accuracy: {test_slot_acc}")
print(f"Test Intent Accuracy: {test_intent_acc}")

ValueError: You must call `compile()` before using the model.

### Predict

In [20]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    info = {"intent": intent_names[intent_id], "slots": {}}

    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([re.sub(r'^[BI]-', '', slot_names[s]) for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)

    # process sequence output for slots
    for token, slot_id in zip(tokens, slot_ids):
      # add all to out_dict
      slot_name = slot_names[slot_id]

      if slot_name == "[PAD]":
        continue

      # collect tokens
      collected_tokens = [token]
      idx = tokens.index(token)

      # see if it starts with ##
      # then it belongs to the previous token
      if token.startswith("##"):

        # check if the token already exists or not
        if tokens[idx - 1] not in out_dict[slot_name]:
          collected_tokens.insert(0, tokens[idx - 1])

      # add collected tokens to slots
      out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
      tokens = out_dict[slot_name]
      slot_value = tokenizer.convert_tokens_to_string(tokens)

      info["slots"][slot_name] = slot_value.strip()

    return info

In [29]:
#loaded_model = tf.keras.models.load_model('model_epoch_18.keras', custom_objects={'JointIntentAndSlotFillingModel': JointIntentAndSlotFillingModel})

print(nlu("could you please change my reservation on december twenty first originally scheduled for twelve thirty in the morning to now be at four thirty five pm", tokenizer, joint_model, intent_names, slot_names))

{'intent': 'add_res', 'slots': {'NAME': 'thirty', 'ITEM': 'could you please reservation december originally scheduled twelve thirty in morning at four five', 'DRINK': '[CLS] change twenty', 'DATE': 'my on for the now be', 'SIZE': 'first to pm', 'NUMBER': '[SEP]'}}
